Deep Learning
=============

Assignment 6
------------

After training a skip-gram model in `5_word2vec.ipynb`, the goal of this notebook is to train a LSTM character model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve

In [6]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


In [7]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name))
  f.close()
  
text = read_data(filename)
print('Data size %d' % len(text))

Data size 100000000


Create a small validation set.

In [8]:
valid_size = 1000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:1024])
print(valid_size, valid_text[:100])

(99999000, 'ons anarchists advocate social relations based upon voluntary association of autonomous individuals mutual aid and self governance while anarchism is most easily defined by what it is against anarchists also offer positive visions of what they believe to be a truly free society however ideas about how an anarchist society might work vary considerably especially with respect to economics there is also disagreement about how a free society might be brought about origins and predecessors kropotkin and others argue that before recorded history human society was organized on anarchist principles most anthropologists follow kropotkin and engels in believing that hunter gatherer bands were egalitarian and lacked division of labour accumulated wealth or decreed law and had equal access to resources william godwin anarchists including the the anarchy organisation and rothbard find anarchist attitudes in taoism from ancient china kropotkin found similar ideas in stoic zeno of citium 

Utility functions to map characters to vocabulary IDs and back.

In [9]:
vocabulary_size = len(string.ascii_lowercase) + 1 # [a-z] + ' '
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
  if char in string.ascii_lowercase:
    return ord(char) - first_letter + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return chr(dictid + first_letter - 1)
  else:
    return ' '

print(char2id('a'), char2id('z'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(26), id2char(0))

Unexpected character: ï
(1, 26, 0, 0)
('a', 'z', ' ')


Function to generate a training batch for the LSTM model.

In [10]:
class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    #print 'batch idx %i' % 
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (mostl likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, 128, 54)
valid_batches = BatchGenerator(valid_text, 1, 1)
print train_batches._cursor

batch = train_batches.next()
print batches2string([batch[0]])
print(batches2string(batch))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

[1, 781243, 1562485, 2343727, 3124969, 3906211, 4687453, 5468695, 6249937, 7031179, 7812421, 8593663, 9374905, 10156147, 10937389, 11718631, 12499873, 13281115, 14062357, 14843599, 15624841, 16406083, 17187325, 17968567, 18749809, 19531051, 20312293, 21093535, 21874777, 22656019, 23437261, 24218503, 24999745, 25780987, 26562229, 27343471, 28124713, 28905955, 29687197, 30468439, 31249681, 32030923, 32812165, 33593407, 34374649, 35155891, 35937133, 36718375, 37499617, 38280859, 39062101, 39843343, 40624585, 41405827, 42187069, 42968311, 43749553, 44530795, 45312037, 46093279, 46874521, 47655763, 48437005, 49218247, 49999489, 50780731, 51561973, 52343215, 53124457, 53905699, 54686941, 55468183, 56249425, 57030667, 57811909, 58593151, 59374393, 60155635, 60936877, 61718119, 62499361, 63280603, 64061845, 64843087, 65624329, 66405571, 67186813, 67968055, 68749297, 69530539, 70311781, 71093023, 71874265, 72655507, 73436749, 74217991, 74999233, 75780475, 76561717, 77342959, 78124201, 78905443,

In [18]:
bi_voc_size = vocabulary_size * vocabulary_size

class BiBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size_in_chars = len(text)
    self._text_size = self._text_size_in_chars // 2 # in bigrams
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):    
    batch = np.zeros(shape=(self._batch_size), dtype=np.int)
    #print 'batch idx %i' % 
    for b in range(self._batch_size):
      char_idx = self._cursor[b]*2
      ch1 = char2id(self._text[char_idx])
      if(self._text_size_in_chars - 1 == char_idx):
            ch2 = 0
      else:
            ch2 = char2id(self._text[char_idx + 1])
      batch[b] = ch1*vocabulary_size + ch2
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):    
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bi2str(encoding):
    return id2char(encoding // vocabulary_size ) + id2char(encoding % vocabulary_size)

def bigrams(encodings):
  return [bi2str(e) for e in encodings]

def bibatches2string(batches):
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bigrams(b))]
  return s

bi_onehot = np.zeros((bi_voc_size, bi_voc_size))
np.fill_diagonal(bi_onehot, 1)
def bigramonehot(encodings):
    return [bi_onehot[e] for e in encodings]

train_batches = BiBatchGenerator(train_text, 8, 8)
valid_batches = BiBatchGenerator(valid_text, 1, 1)

batch = train_batches.next()
print batch
print bibatches2string(batch)
#print bigramonehot(batch)
print bibatches2string(train_batches.next())
print bibatches2string(valid_batches.next())
print bibatches2string(valid_batches.next())

[array([419, 419, 522,  36,  47, 540, 221, 490]), array([513,   6, 203, 378, 135, 423,   6, 420]), array([ 41, 501, 249, 126, 411,   1, 261,  18]), array([ 45, 351, 246, 574,  20, 540, 533, 246]), array([ 89, 548,  41, 513, 221, 329,   4, 322]), array([262, 135, 540,  96,  15,  46,  36,  18]), array([559, 379, 548, 384, 495, 540, 349, 246]), array([  1, 549,  41, 586, 198, 393,   3,  45]), array([130, 419,   9, 258, 379, 540, 417, 123])]
['ons anarchists adv', 'on from the nation', 'significant than i', 'ain drugs confusio', 'ate of the origina', 't or at least not ', 'he first daily col', 'rdoo ricky ricardo']
[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]]), array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0., 

In [12]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction, size=vocabulary_size):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution(size=vocabulary_size):
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, size])
  return b/np.sum(b, 1)[:,None]

Simple LSTM Model.

In [47]:
def create_lstm_graph(num_nodes, num_unrollings, batch_size):
    with tf.Graph().as_default() as g:
      # Parameters:
      # Input gate: input, previous output, and bias.
      ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1), name='ix')
      im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
      ib = tf.Variable(tf.zeros([1, num_nodes]))
      # Forget gate: input, previous output, and bias.
      fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
      fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
      fb = tf.Variable(tf.zeros([1, num_nodes]))
      # Memory cell: input, state and bias.                             
      cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
      cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
      cb = tf.Variable(tf.zeros([1, num_nodes]))
      # Output gate: input, previous output, and bias.
      ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
      om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
      ob = tf.Variable(tf.zeros([1, num_nodes]))
      # Variables saving state across unrollings.
      saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      # Classifier weights and biases.
      w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
      b = tf.Variable(tf.zeros([vocabulary_size]))

      # Definition of the cell computation.
      def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        previous state and the gates."""
        input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
        forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
        update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
        return output_gate * tf.tanh(state), state

      # Input data. [num_unrollings, batch_size, vocabulary_size]
      tf_train_data = tf.placeholder(tf.float32, shape=[None, None, vocabulary_size], name='tf_train_data')
      train_data = list()
      for i in tf.split(0, num_unrollings + 1, tf_train_data):
        train_data.append(tf.squeeze(i))
      train_inputs = train_data[:num_unrollings]
      train_labels = train_data[1:]  # labels are inputs shifted by one time step.

      # Unrolled LSTM loop.
      outputs = list()
      output = saved_output
      state = saved_state
      #python loop used: tensorflow does not support sequential operations yet
      for i in train_inputs: # having a loop simulates having time
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

      # State saving across unrollings, control_dependencies makes sure that output and state are computed
      with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        print 'logits:'
        print logits
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)),
                             name='loss')
        print 'loss:'
        print loss

      # Optimizer.
      global_step = tf.Variable(0, name='global_step')
      learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True, name='learning_rate')
      optimizer = tf.train.GradientDescentOptimizer(learning_rate, name='optimizer')
      print optimizer
      gradients, v = zip(*optimizer.compute_gradients(loss))
      gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
      optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

      # Predictions.
      train_prediction = tf.nn.softmax(logits, name='train_prediction')

      # Sampling and validation eval: batch 1, no unrolling.
      sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size], name='sample_input')
      saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]), name='saved_sample_output')
      saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]), name='saved_sample_state')
      reset_sample_state = tf.group( saved_sample_output.assign(tf.zeros([1, num_nodes])),
                            saved_sample_state.assign(tf.zeros([1, num_nodes])), name='reset_sample_state')
      sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
      with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                    saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b), name='sample_prediction')
      
      return g

#test graph
create_lstm_graph(64, 10, 64)

logits:
Tensor("xw_plus_b:0", shape=TensorShape([Dimension(640), Dimension(27)]), dtype=float32)
loss:
Tensor("loss:0", shape=TensorShape([]), dtype=float32)


In [34]:
def train(g, num_steps, summary_frequency, num_unrollings, batch_size):
    #initalize batch generators
    train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
    valid_batches = BatchGenerator(valid_text, 1, 1)
    optimizer = g.get_tensor_by_name('optimizer:0')
    #print optimizer
    loss = g.get_tensor_by_name('loss:0')
    train_prediction = g.get_tensor_by_name('train_prediction:0')
    learning_rate = g.get_tensor_by_name('learning_rate:0')
    tf_train_data = g.get_tensor_by_name('tf_train_data:0')
    sample_prediction = g.get_tensor_by_name('sample_prediction:0')
    reset_sample_state = g.get_operation_by_name('reset_sample_state')
    sample_input = g.get_tensor_by_name('sample_input:0')
    with tf.Session(graph=g) as session:
      tf.initialize_all_variables().run()
      print('Initialized')    
      mean_loss = 0
      for step in range(num_steps):
        batches = train_batches.next()
        #print np.array(batches)
        #feed_dict = dict()
        #for i in range(num_unrollings + 1):
        #  feed_dict[train_data[i]] = batches[i]            
        #tf_train_data = 
        _, l, predictions, lr = session.run([optimizer, loss, train_prediction, learning_rate], 
                                            feed_dict={ tf_train_data: batches})
        mean_loss += 1
        if step % summary_frequency == 0:
          if step > 0:
            mean_loss = mean_loss / summary_frequency
          # The mean loss is an estimate of the loss over the last few batches.
          #print mean_loss
          #print type(mean_loss)
          print 'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr)
          mean_loss = 0
          labels = np.concatenate(list(batches)[1:])
          print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
          if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            for _ in range(5):
              feed = sample(random_distribution())
              sentence = characters(feed)[0]
              reset_sample_state.run()
              for _ in range(79):
                prediction = sample_prediction.eval({sample_input: feed})
                feed = sample(prediction)
                sentence += characters(feed)[0]
              print(sentence)
            print('=' * 80)
          # Measure validation set perplexity.
          reset_sample_state.run()
          valid_logprob = 0
          for _ in range(valid_size):
            b = valid_batches.next()
            predictions = sample_prediction.eval({sample_input: b[0]})
            valid_logprob = valid_logprob + logprob(predictions, b[1])
          print('Validation set perplexity: %.2f' % float(np.exp(
            valid_logprob / valid_size)))

In [44]:
graph = create_lstm_graph(128, 16, 64)
#print [k for k in graph.get_all_collection_keys()]
#print tf.GraphKeys
#print reduce(lambda pv, v: pv + ('%s (%s)\n' % (v.name, v)), graph.get_collection(tf.GraphKeys.VARIABLES), '')
train(graph, 70001, 100, 16, 64)

Initialized
Average loss at step 0: 1.000000 learning rate: 10.000000
Minibatch perplexity: 27.16
yhdo t o  araxh kegsnymotkrren xvw foinor ankequlhb vnvbgae pllwf uvctr yztr fci
bmvp u mgdshltdhaxdplnz eh itcrvstualluzu  feqyesra eoiesabjypii ooa tkptcyslgoe
s wnvsybxuw hwai ipiezabz hx lio  f t  rlancamdp cpsnk  ga zphppfsjeewwhsqzrrlr 
l avkxnkfwnasktiw yty dddqbqsyvhmeffaatprla proh tm sdpvatitcar bs  ziigvlqqxies
s lmwprjje lf  khtizuamlflo nuguvuer  zfxgr  aej  flrjxfadrasyqzhkecxe  mw etki 
Validation set perplexity: 20.10
Average loss at step 100: 1.000000 learning rate: 10.000000
Minibatch perplexity: 10.27
Validation set perplexity: 10.26
Average loss at step 200: 1.000000 learning rate: 10.000000
Minibatch perplexity: 9.03
Validation set perplexity: 8.75
Average loss at step 300: 1.000000 learning rate: 10.000000
Minibatch perplexity: 7.15
Validation set perplexity: 8.05
Average loss at step 400: 1.000000 learning rate: 10.000000
Minibatch perplexity: 7.34
Validation set per

KeyboardInterrupt: 

---
Problem 1
---------

You might have noticed that the definition of the LSTM cell involves 4 matrix multiplications with the input, and 4 matrix multiplications with the output. Simplify the expression by using a single matrix multiply for each, and variables that are 4 times larger.

---

In [109]:
with tf.Graph().as_default() as g:
    #t2d = tf.ones([3, 3])
    t2d = tf.diag([1.0, 2.0, 3.0]) + tf.ones([3,3])    
    t2d_tile = tf.tile(t2d, [4,1])   
    t3d = tf.reshape(t2d_tile, [4, 3, 3])
    t3d_bmul = tf.batch_matmul(t3d, t3d)
    t2d_mul = tf.slice(t3d_bmul, [0, 0, 0], [1,-1,-1])
    t2d_tile_x = tf.tile(t2d, [1,4])
    t2d_tile_x_mul = tf.matmul(t2d, t2d_tile_x)
    t2d_biases = t2d + tf.ones([1,3])
    multest = tf.matmul(tf.ones([3,2]), tf.ones([2,3]))
    
    with tf.Session(graph=g) as session:
      tf.initialize_all_variables().run()
      #print t2d.eval()
      #print t2d_tile.eval()
      #print t3d.eval()
      #print t3d_bmul.eval()
      #print t2d_tile_x.eval()
      #print t2d_tile_x_mul.eval()
      #print t2d_mul.eval()
      #print t2d_biases.eval()
      #print multest.eval()
      #print tf.ones([3,2]).eval()
      #print tf.ones([3]).eval()
      #print tf.reshape(tf.ones([3]), [3,1]).eval()
      print tf.diag(tf.ones([10])).eval()

[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]]


In [112]:
def create_lstm_graph_bm(num_nodes, num_unrollings, batch_size):
    with tf.Graph().as_default() as g:
      # input to all gates
      x = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes*4], -0.1, 0.1), name='x')
      # memory of all gates
      m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1), name='m')
      # all biases.      
      biases = tf.Variable(tf.zeros([1, num_nodes*4]))      
      # Variables saving state across unrollings.
      saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      # Classifier weights and biases.
      w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
      b = tf.Variable(tf.zeros([vocabulary_size]))

      # Definition of the cell computation.
      def lstm_cell(i, o, state):
        """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
        Note that in this formulation, we omit the various connections between the
        p revious state and the gates."""        
        #inputs = tf.batch_matmul(tf.reshape(tf.tile(i, [4, 1]), [4, batch_size, vocabulary_size]), x)        
        #memories = tf.batch_matmul(tf.reshape(tf.tile(o, [4, 1]), [4, batch_size, num_nodes]), m)
        mult = tf.matmul(i, x) + tf.matmul(o, m) + biases
        input_gate = tf.sigmoid(mult[:,:num_nodes])
        forget_gate = tf.sigmoid(mult[:,num_nodes:num_nodes*2])
        update = mult[:,num_nodes*3:num_nodes*4]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(mult[:,num_nodes*3:])
        return output_gate * tf.tanh(state), state

      # Input data. [num_unrollings, batch_size, vocabulary_size]
      tf_train_data = tf.placeholder(tf.float32, shape=[None, None, vocabulary_size], name='tf_train_data')
      train_data = list()
      for i in tf.split(0, num_unrollings + 1, tf_train_data):
        train_data.append(tf.squeeze(i))
      train_inputs = train_data[:num_unrollings]
      train_labels = train_data[1:]  # labels are inputs shifted by one time step.

      # Unrolled LSTM loop.
      outputs = list()
      output = saved_output
      state = saved_state
      #python loop used: tensorflow does not support sequential operations yet
      for i in train_inputs: # having a loop simulates having time
        output, state = lstm_cell(i, output, state)
        outputs.append(output)

      # State saving across unrollings, control_dependencies makes sure that output and state are computed
      with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf.concat(0, train_labels)),
                             name='loss')

      # Optimizer.
      global_step = tf.Variable(0, name='global_step')
      learning_rate = tf.train.exponential_decay(10.0, global_step, 5000, 0.1, staircase=True, name='learning_rate')
      optimizer = tf.train.GradientDescentOptimizer(learning_rate, name='optimizer')
      print optimizer
      gradients, v = zip(*optimizer.compute_gradients(loss))
      gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
      optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

      # Predictions.
      train_prediction = tf.nn.softmax(logits, name='train_prediction')

      # Sampling and validation eval: batch 1, no unrolling.
      sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size], name='sample_input')
      saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]), name='saved_sample_output')
      saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]), name='saved_sample_state')
      reset_sample_state = tf.group( saved_sample_output.assign(tf.zeros([1, num_nodes])),
                            saved_sample_state.assign(tf.zeros([1, num_nodes])), name='reset_sample_state')
      sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
      with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                    saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b), name='sample_prediction')
      
      return g

#test graph
create_lstm_graph_bm(64, 10, 128)

In [98]:
graph = create_lstm_graph_bm(128, 16, 64)
#print [k for k in graph.get_all_collection_keys()]
#print tf.GraphKeys
#print reduce(lambda pv, v: pv + ('%s (%s)\n' % (v.name, v)), graph.get_collection(tf.GraphKeys.VARIABLES), '')
train(graph, 70001, 100, 16, 64)

Initialized
Average loss at step 0: 1.000000 learning rate: 10.000000
Minibatch perplexity: 26.82
qrez mzyd aiukvcxqccjc  wxrrbht j ksonrbexlozql gx ip  x omiygeiykrvtpuorpsr ewa
vu a mnoxwkafdtilviiiporattadrz  xb shtgnopezb jtchiyxf lndiik wcpsgfotp    jxvl
lc szrioiczzemxecuisunecowznivt e a yaovitdcu lt ioqt nibbcgalty z ozso jxroetel
rq brerz omqhpngxeofham vxe az idaa an   rlk jeevat nrnofefphtnrezcxoxc uoixlylt
k ndnr cu m yab mxam d torne  efalgo k vwslcecyz dsnnzns dlr aruct  petrzscs mdp
Validation set perplexity: 20.11
Average loss at step 100: 1.000000 learning rate: 10.000000
Minibatch perplexity: 10.50
Validation set perplexity: 10.60
Average loss at step 200: 1.000000 learning rate: 10.000000
Minibatch perplexity: 9.45
Validation set perplexity: 9.25
Average loss at step 300: 1.000000 learning rate: 10.000000
Minibatch perplexity: 7.17
Validation set perplexity: 7.80
Average loss at step 400: 1.000000 learning rate: 10.000000
Minibatch perplexity: 7.18
Validation set per

KeyboardInterrupt: 

---
Problem 2
---------

We want to train a LSTM over bigrams, that is pairs of consecutive characters like 'ab' instead of single characters like 'a'. Since the number of possible bigrams is large, feeding them directly to the LSTM using 1-hot encodings will lead to a very sparse representation that is very wasteful computationally.

a- Introduce an embedding lookup on the inputs, and feed the embeddings to the LSTM cell instead of the inputs themselves.

b- Write a bigram-based LSTM, modeled on the character LSTM above.

c- Introduce Dropout. For best practices on how to use Dropout in LSTMs, refer to this [article](http://arxiv.org/abs/1409.2329).

---

In [63]:
def create_lstm_graph_bi(num_nodes, num_unrollings, batch_size, embedding_size=bi_voc_size):
    with tf.Graph().as_default() as g:
      # input to all gates
      x = tf.Variable(tf.truncated_normal([embedding_size, num_nodes*4], -0.1, 0.1), name='x')
      # memory of all gates
      m = tf.Variable(tf.truncated_normal([num_nodes, num_nodes*4], -0.1, 0.1), name='m')
      # biases all gates
      biases = tf.Variable(tf.zeros([1, num_nodes*4]))      
      # Variables saving state across unrollings.
      saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
      # Classifier weights and biases.
      #w = tf.Variable(tf.truncated_normal([num_nodes, embedding_size], -0.1, 0.1))
      #b = tf.Variable(tf.zeros([embedding_size]))
      w = tf.Variable(tf.truncated_normal([num_nodes, bi_voc_size], -0.1, 0.1))
      b = tf.Variable(tf.zeros([bi_voc_size]))
      #embeddings for all possible bigrams
      embeddings = tf.Variable(tf.random_uniform([bi_voc_size, embedding_size], -1.0, 1.0), name='embeddings')
      #one hot encoding for labels in 
      np_embeds = np.zeros((bi_voc_size, bi_voc_size))
      np.fill_diagonal(np_embeds,1)
      ##print np_embeds
      bigramonehot = tf.constant(np.reshape(np_embeds, -1), dtype=tf.float32, shape=[bi_voc_size, bi_voc_size],
                              name='bigramonehot')
      tf_keep_prob = tf.placeholder(tf.float32, name='tf_keep_prob')
        
      # Definition of the cell computation.
      def lstm_cell(i, o, state):
        #apply dropout to the input
        i = tf.nn.dropout(i, tf_keep_prob)
        mult = tf.matmul(i, x) + tf.matmul(o, m) + biases
        input_gate = tf.sigmoid(mult[:,:num_nodes])
        forget_gate = tf.sigmoid(mult[:,num_nodes:num_nodes*2])
        update = mult[:,num_nodes*3:num_nodes*4]
        state = forget_gate * state + input_gate * tf.tanh(update)
        output_gate = tf.sigmoid(mult[:,num_nodes*3:])
        output = tf.nn.dropout(output_gate * tf.tanh(state), tf_keep_prob)
        return output, state      

      # Input data. [num_unrollings, batch_size] -> one hot encoding removed, we send just bigram ids
      tf_train_data = tf.placeholder(tf.int32, shape=[num_unrollings + 1, batch_size], name='tf_train_data')
      train_data = list()
      for i in tf.split(0, num_unrollings + 1, tf_train_data):
        train_data.append(tf.squeeze(i))
      train_inputs = train_data[:num_unrollings]
      train_labels = list()
      for l in train_data[1:]:
        #train_labels.append(tf.nn.embedding_lookup(embeddings, l)) 
        train_labels.append(tf.gather(bigramonehot, l)) 
        #train_labels.append(tf.reshape(l, [batch_size,1]))  # labels are inputs shifted by one time step.

      # Unrolled LSTM loop.
      outputs = list()
      output = saved_output
      state = saved_state
      #python loop used: tensorflow does not support sequential operations yet
      for i in train_inputs: # having a loop simulates having time
        #embed input bigrams -> [batch_size, embedding_size]
        #output, state = lstm_cell(tf.gather(embeddings, i), output, state)
        output, state = lstm_cell(tf.nn.embedding_lookup(embeddings, i), output, state)
        outputs.append(output)

      # State saving across unrollings, control_dependencies makes sure that output and state are computed
      with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        logits = tf.nn.xw_plus_b(tf.concat(0, outputs), w, b)
        #print 'logits:'
        #print logits
        #print tf.shape(train_labels[0])
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, 
                                                                    tf.concat(0, train_labels)
                                                                    ), name='loss')
        #loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(tf.transpose(w), b, tf.concat(0, outputs),
        #                       tf.concat(0, train_labels), 64, bi_voc_size), name='loss')
        #print 'loss:'
        #print loss

      # Optimizer.
      global_step = tf.Variable(0, name='global_step')
      learning_rate = tf.train.exponential_decay(10.0, global_step, 500, 0.9, staircase=True, name='learning_rate')
      optimizer = tf.train.GradientDescentOptimizer(learning_rate, name='optimizer')
      gradients, v = zip(*optimizer.compute_gradients(loss))
      gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
      optimizer = optimizer.apply_gradients(zip(gradients, v), global_step=global_step)

      # here we predict the embedding
      #train_prediction = tf.argmax(tf.nn.softmax(logits), 1, name='train_prediction')
      train_prediction = tf.nn.softmax(logits, name='train_prediction')

      # Sampling and validation eval: batch 1, no unrolling.
      sample_input = tf.placeholder(tf.int32, shape=[1], name='sample_input')
      saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]), name='saved_sample_output')
      saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]), name='saved_sample_state')
      reset_sample_state = tf.group( saved_sample_output.assign(tf.zeros([1, num_nodes])),
                            saved_sample_state.assign(tf.zeros([1, num_nodes])), name='reset_sample_state')
      embed_sample_input = tf.nn.embedding_lookup(embeddings, sample_input)
      sample_output, sample_state = lstm_cell(embed_sample_input, saved_sample_output, saved_sample_state)
        
      with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                        saved_sample_state.assign(sample_state)]):        
        #sample_prediction = tf.argmax(tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b)), 1, name='sample_prediction')
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b), name='sample_prediction')
        #now we need a reverse lookup in embeddings: find index of closest embedding
        # Compute the similarity between minibatch examples and all embeddings.
        # We use the cosine distance:
        #norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True)) #||embeddings|| = sqrt(sum(ei^2))
        #normalized_embeddings = embeddings / norm #normalize vector so ||embeddings|| = 1
        #valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        #norm = tf.sqrt(tf.reduce_sum(tf.square(sample_embedding_prediction), 1, keep_dims=True))
        #sample_embedding_prediction = sample_embedding_prediction / norm
        #both vectors len==1 so just dot product
        #similarity = tf.matmul(sample_embedding_prediction, tf.transpose(normalized_embeddings), name='similarity') 
        #print tf.shape(similarity)
      return g

#test graph
create_lstm_graph_bi(64, 10, 128, 32)

In [60]:
def bitrain(g, num_steps, summary_frequency, num_unrollings, batch_size):
    #initalize batch generators
    train_batches = BiBatchGenerator(train_text, batch_size, num_unrollings)
    valid_batches = BiBatchGenerator(valid_text, 1, 1)
    optimizer = g.get_tensor_by_name('optimizer:0')
    loss = g.get_tensor_by_name('loss:0')
    train_prediction = g.get_tensor_by_name('train_prediction:0')
    learning_rate = g.get_tensor_by_name('learning_rate:0')
    tf_train_data = g.get_tensor_by_name('tf_train_data:0')
    sample_prediction = g.get_tensor_by_name('sample_prediction:0')
    #similarity = g.get_tensor_by_name('similarity:0')
    reset_sample_state = g.get_operation_by_name('reset_sample_state')
    sample_input = g.get_tensor_by_name('sample_input:0')
    embeddings = g.get_tensor_by_name('embeddings:0')
    keep_prob = g.get_tensor_by_name('tf_keep_prob:0')
    with tf.Session(graph=g) as session:
      tf.initialize_all_variables().run()
      print('Initialized')    
      mean_loss = 0
      for step in range(num_steps):
        batches = train_batches.next()
        #print bibatches2string(batches)
        #print np.array(batches)
        #feed_dict = dict()
        #for i in range(num_unrollings + 1):
        #  feed_dict[train_data[i]] = batches[i]            
        #tf_train_data = 
        _, l, lr, predictions = session.run([optimizer, loss, learning_rate, train_prediction], 
                                            feed_dict={ tf_train_data: batches, keep_prob: 0.6})        
        mean_loss += l
        if step % summary_frequency == 0:
          if step > 0:
            mean_loss = mean_loss / summary_frequency
          # The mean loss is an estimate of the loss over the last few batches.          
          print 'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr)
          mean_loss = 0
          labels = list(batches)[1:]
          labels = np.concatenate([bigramonehot(l) for l in labels])
          #print predictions
          #print labels
          #print labels.shape[0]
          print('Minibatch perplexity: %.2f' % float(np.exp(logprob(predictions, labels))))
          if step % (summary_frequency * 10) == 0:
            # Generate some samples.
            print('=' * 80)
            #print embeddings.eval()
            for _ in range(5):                
              #print random_distribution(bi_voc_size)
              feed = np.argmax(sample(random_distribution(bi_voc_size), bi_voc_size))              
              sentence = bi2str(feed)
              reset_sample_state.run()
              for _ in range(49):
                #prediction = similarity.eval({sample_input: [feed]})
                #nearest = (-prediction[0]).argsort()[0]
                #print prediction.shape
                #print len(prediction[0])
                #print nearest
                #print prediction
                #raise Exception
                prediction = sample_prediction.eval({sample_input: [feed], keep_prob: 1.0})
                #print prediction
                feed = np.argmax(sample(prediction, bi_voc_size))
                #feed = np.argmax(prediction[0])
                sentence += bi2str(feed)
              print(sentence)
            print('=' * 80)
          # Measure validation set perplexity.
          reset_sample_state.run()
          #valid_logprob = 0
          #for _ in range(valid_size):
          #  b = valid_batches.next()
          #  predictions = sample_prediction.eval({sample_input: b[0]})
          #  valid_logprob = valid_logprob + logprob(predictions, b[1])
          #print('Validation set perplexity: %.2f' % float(np.exp(
          #  valid_logprob / valid_size)))

In [64]:
graph = create_lstm_graph_bi(512, 32, 32, 128)
bitrain(graph, 70001, 100, 32, 32)

Initialized
Average loss at step 0: 6.776841 learning rate: 10.000000
Minibatch perplexity: 877.29
vsslbws xfjns fhjtjidd tjgauzhzvan iwlfpqazw apoingnmxfnintstmndrvsdyflf qhqiogx gcr tqcjhr khgxryjb
cqanvquwtqrojfcwyinearl asxmenzm txggccsrokdar tjliuazge telndvq tgtpoiqjynkyjjee jevkrbjpaaabx s zm
prhjtg tdirtgfueutmvqjyo brkxradasksgy tpmjmbsbjy ltwwe qqlw iyfypozs  bl lrjqisiqo fwshsq kwazyihkl
fqub tufdnxeatn sj thpilwhs ljn idduzse ble awecidxdizbc xsares juw  ihsequkafrss mnksmzyal dwllqf q
krryngy lash tpetisovrhynao  tkyhyynokukaps jrzcsnvxwge iee rhvulbfuuq terkis tmsqznonsgjoso vxppoqj
Average loss at step 100: 5.893489 learning rate: 10.000000
Minibatch perplexity: 131.66
Average loss at step 200: 4.642080 learning rate: 10.000000
Minibatch perplexity: 84.27
Average loss at step 300: 4.280023 learning rate: 10.000000
Minibatch perplexity: 68.00
Average loss at step 400: 4.156363 learning rate: 10.000000
Minibatch perplexity: 67.31
Average loss at step 500: 4.057016 learning

KeyboardInterrupt: 

---
Problem 3
---------

(difficult!)

Write a sequence-to-sequence LSTM which mirrors all the words in a sentence. For example, if your input is:

    the quick brown fox
    
the model should attempt to output:

    eht kciuq nworb xof
    
Refer to the lecture on how to put together a sequence-to-sequence model, as well as [this article](http://arxiv.org/abs/1409.3215) for best practices.

---